In [1]:
import numpy as np
import random
import pickle
import gzip

In [2]:
class Network(object):

    #This method is automatically called each time a new instance of the class is created
    def __init__(self, sizes):
        
            #Parameter 'sizes' is a list that describes the Artificial Neural Network
            #Length of 'sizes' is the number of layers in the ANN
            #Each number in the list corresponds to the number of neurons
            #in  that layer
            #Use standard normal distribution to initialize biases and weights
            #Normal function with mean of 0 and variance of 1
        
            #First layer does not recieve a bias as it is the input layer
            #Last layer is the output layer
        
            #For example, if 'sizes' = [3, 4, 2]
            #That means a network with with 3 inputs in first layer
            #4 neurons in hidden layer
            #2 outputs in the output layer
        
            #Every layer between input layer and output layer is a hidden layer
        
            #Number of layers is the length of the 'sizes' list
            #Creating instance level of variable 'num_layers'
        self.num_layers = len(sizes)
        
            #Creating instance level of list 'sizes'
        self.sizes = sizes
        
            #Set biases for every layer except the first, specified by sizes[1:]
            #Using standard normal distribution, np.random.randn(), where mean is 0 and variance is 1
            #Each neuron in an activation layer has exactly one corresponding bias
            #Each layer is represented by an array
            #Each bias in that layer is a value in the array corresponding to its layer
            #The position of each bias in the array corresponds to its position in the layer
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        
            #Set weights for each connection
            #Each neuron in a layer receives a weight for each neuron in the next layer
            #Here zip() basically gives us the dimensions of weight matrix
            #Each 
        self.weights = [np.random.randn(y, x) for x, y in zip(sizes[:-1], sizes[1:])]
        
        
    def feedforward(self, a):
        """Return the output of the network if ``a`` is input."""
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a)+b)
        return a

    
    def SGD(self, training_data, epochs, mini_batch_size, eta, test_data=None):
        """Train the neural network using mini-batch stochastic
        gradient descent.  The ``training_data`` is a list of tuples
        ``(x, y)`` representing the training inputs and the desired
        outputs.  The other non-optional parameters are
        self-explanatory.  If ``test_data`` is provided then the
        network will be evaluated against the test data after each
        epoch, and partial progress printed out.  This is useful for
        tracking progress, but slows things down substantially."""
        
        
            #If you have test data, find the length of it/how many pairs there are
        if test_data:
            n_test = len(test_data)
            
            #Find length of training data/how many pairs there are
        n = len(training_data)
        for j in range(epochs):
            
                #Shuffle the pairs tuples of input/output to keep ANN from
                #learning a pattern in way inputs come up
            random.shuffle(training_data)
            
                #Splits training data into (len(training_data)/mini_batch_size) piles
                #Basically splits training_data into piles (batches) of mini_batch_size inputs
                #Splits training data into a list of lists of tuples of ([input], [output]) ndarrays
                #Not putting each I/O pair into a mini batch individually, instead puts 10 in list
                #then slids up and puts 10 more in
            mini_batches = [training_data[k:k+mini_batch_size]
                            for k in range(0, n, mini_batch_size)]
            
                #Method to correct the ANN.  Pass one mini batch worth of data to
                #update mini batch function.  Do that until you are out of mini batches
            for mini_batch in mini_batches:                
                self.update_mini_batch(mini_batch, eta)
                
                #If there is testing data, test after each epoch
                #Maybe shuffle testing data before testing
            #There is no learning from tests, only results
            if test_data:
                print("Epoch {0}: {1} / {2}".format((j + 1), self.evaluate(test_data), n_test))
            else:
                print("Epoch {0} complete".format(j + 1))

                
    def update_mini_batch(self, mini_batch, eta):
        """Update the network's weights and biases by applying
        gradient descent using backpropagation to a single mini batch.
        The ``mini_batch`` is a list of tuples ``(x, y)``, and ``eta``
        is the learning rate."""
        
            #Create ndarrays based on size and number of layers in network
            #Fill the arrays with zeros. Put them in lists, one list for
            #biases and one for weights. Every time we update mini batch
            #Nabla because that's the name of the Greek symbol for gradient
            #Arrays for gradients
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        
            #Takes the inputs and outputs of one mini-batch, feeds them into backpropagation
            #In this case, mini_batch is a set of 10 I/O tuples
            #This loop takes each I/O pair in the mini batch one at a time
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
            
        #'eta' is how big each step in our stochastic gradient descent is
        self.weights = [w-(eta/len(mini_batch))*nw
                        for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b-(eta/len(mini_batch))*nb
                       for b, nb in zip(self.biases, nabla_b)]

        
    def backprop(self, x, y):
        
            #Returns tuple '([nabla_b], [nabla_w])', should be reminiscent of gradient for cost function 
        
            #Make list of ndarrays for biases, fill them with zeros
            #Make list of ndarrays for weights, fill them with zeros
            #Arrays for gradients
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        
            #'x' is the input values array.  'y' is the output values vector
            #Copy input values from 'x' to 'activation'
            #Activation layer is the current layer being fed through the ANN
        activation = x
        
            #List to store each activation layer
            #Literally the list of every layer and its values for each run
            #Copy input values from 'x' to the list
        activations = [x]
        
            #Create empty list to store z vectors. List of past activation layers. Non-sigmoid functioned
            #layers.  Add vectors here after they have been weighted and biased, but before sigmoid.
        zs = []
    
            #Actual neural part with multiplying weights and adding bias
            #For length of 'zip(self.biases, self.weights)' do the looop
            #Zip weights and biases together, pair up sets of weights and biases in tuples
            #Dot product weights with inputs ('activation'), add biases to
            #resultant vectors (z-vector)
            #append 'zs[]' with resulting vector.  Pass that z-vector to sigmoid function so we have output
            #that lies within our constraints (0,1).  Put new new, current activation layer into 'activations' list
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation)+b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
            
            #Passing most recent output layer from list of activation layers and 
            #to 'cost_derivative' method
        delta = self.cost_derivative(activations[-1], y) * sigmoid_prime(zs[-1])
        
            #Put change of last 
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        
            #Start loop at second to last layer, layer closest to output layer.
        for l in range(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
            
            #Return gradient vector components of cost function
        return (nabla_b, nabla_w)
    
    def evaluate(self, test_data):
            #Tell how many correct answers network got
        
            #Keep feeding inputs in, no learning during tests
            #The ANN's answer is the highest value in their output after sigmoid function
            #When we take this 'argmax' takes the highest value of our x outputs
        test_results = [(np.argmax(self.feedforward(x)), y) for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)

    
    def cost_derivative(self, output_activations, y):
        #Take values in last activation layer, the output layer
        #Subtract their values by 'y' and return that resulting vector
        
        return (output_activations-y)

In [3]:
def sigmoid(z):
        #Function that forces all values of neurons between 0 and 1
        #Applied to each layer after the weights and biases are applied
    return 1.0/(1.0+np.exp(-z))

def sigmoid_prime(z):
    
        #Derivative of sigmoid function, written using the sigmoid function.
        #Not my work
    return sigmoid(z)*(1-sigmoid(z))

In [4]:
def load_data(filename=None):
    
        #Opens the file, unzips the file, unpickles the file
        #Returns data as touple of form ('training_data', 'validation_data', 'test_data')
    #?? Pickled file has the data already broken up into training data, validation data, and testing data
    
    
        #'training_data' is list of 50k tuples
        #Each tuple 2 ndarrays of the form ('X', 'Y')
        #'X' is an ndarray (n-dimensional) array of length 784, (784 dimensional array)
            #representing the 28x28 grid for writing. Basically streched out a 28x28
            #grid to 784x1
        #Each of the 784 elements contains a 1 dimensional ndarray
        #Each of these 1 dimensional ndarrays contains one floating point
            #number between (0,1)
    
        #'Y' is an ndarray of length 10, (10 dimensional array), representing each of
            #the possible digits (0-9)
        #Each of the 10 elements contains a 1 dimensional ndarray
        #Exactly one of these 10 ndarrays of 1 dimenstion contains the floating
            #point number 1.  All others contain 0.
        
        #'X' represents the inputs
        #'Y' represents the correct outputs
    
    
    if not filename:
        filename = 'mnist.pkl.gz'
        
        #Opens file 'mnist.pkl.gz'
        #gzip is for reading and writing 'gzip' files
        #First file is unzipped
    f = gzip.open(filename, 'rb')
    
        #The '.pkl' file extension is a pickle file
        #Pickle files are used to preserve structure of data within them
    training_data, validation_data, test_data = pickle.load(f, encoding='latin1') # Encoding needed for Python 3
    f.close()
    return (training_data, validation_data, test_data)

In [5]:
def load_data_wrapper(filename=None):
    
        #Return a tuple as '("training_data", "validation_data", "testing_data")'    
    
        #'training_data', 'validation_data', 'testing_data' are returned from
        #the method 'load_data' and stored as'tr_d', 'va_d', 'te_d' respectively
    tr_d, va_d, te_d = load_data(filename=filename)
    
        #'tr_d' is a tuple of 2 ndarrays, one containing the 50000 ndarrays
        #of 784 dimensions as inputs.  The other list contains the 50000 ndarrays
        #of 1 dimension containing the integer value of output
        #as in it holds 5, not [0,0,0,0,1]    
    
        #'va_d' is a tuple of length 2.
        #(10k-dimension ndarray, 10k-dimension ndarray)
        #First ndarray has its 10k ndarrays filled with 784 dimension input arrays
        #Second ndarray has its 10k ndarrays filled with the integer value of output
        #to respective input
    
        #'te_d' is tuple of length 2.
        #(10k-dimension ndarray, 10-kdimension ndarray)
        #First ndarray has its 10k ndarrays filled with 784 dimension input arrays
        #Second ndarray has its 10k ndarrays filled with the integer value of output
        #to respective input
    
    
        #Store only the inputs (784 values from (0,1)) from each training exercise
        #as 'training_inputs'.  Is a list of 50000 ndarrays of dimension 784
        #Dynamically shapes input array based on shape of input with 'np.reshape()'
        #Add data, then reshape to 784x1 each iteration
        #Because tr_d[0] is an ndarray, when iterated through with 'x'
    training_inputs = [np.reshape(x, (784, 1)) for x in tr_d[0]]
    
        #Store only the outputs (10 values, exactly one is 1) from each training
        #exercise as 'training_results'.  Is a list of 50000 vectors with 10 elements
        #Dynamically create unit vectors of proper length for solutions
        #by passing the integer value of output and storing a 1 in that position of
        #unit vector.  Add the vector to list of results
    training_results = [vectorized_result(y) for y in tr_d[1]]
    
        #Here zip takes 2 lists, (input and ouput lists) and puts them into tuples
        #of length 2, so tuples look like ([input array], [output array])
        #List of all 50k training exercises
        #Each list is tuple with (input, output), so input is 784 dimension ndarray
        #output is 10 dimensional unit vector
    training_data = list(zip(training_inputs, training_results))

        #List of 10k ndarrays of 784 inputs
    validation_inputs = [np.reshape(x, (784, 1)) for x in va_d[0]]
    
    
        #List of 10k tuples, ([input array], integer value of respective output)
    validation_data = list(zip(validation_inputs, va_d[1]))

        #List of 10k ndarrays worth of input arrays
    test_inputs = [np.reshape(x, (784, 1)) for x in te_d[0]]
    
        #List of 10k tuples, ([input array], integer output)
    test_data = list(zip(test_inputs, te_d[1]))
    return (training_data, validation_data, test_data)

In [6]:
def vectorized_result(j):
        #Returns 10 dimensional unit vector with a 1 in the position that
        #corresponds with the correct digit for the respective input values
        #Change 10 below to a constant to adapt for situations with <10 outputs
        #Accepts integer value, places that integer at that spot in unit vector
    e = np.zeros((10, 1))
    e[j] = 1.0
    return e

In [7]:
#Break data up into its different pieces
training_data, validation_data, test_data = load_data_wrapper()

In [24]:
#Instantiate new network
mathnet = Network([784, 26, 10])

In [ ]:
#Test the network before training

n_test = len(test_data)

print("Pre-training test results", mathnet.evaluate(test_data), n_test)

In [27]:
#Store all of the initial weights and biases
#Write the weights and biases to files

ibiases = mathnet.biases
iweights = mathnet.weights

initial = list(zip(mathnet.biases, mathnet.weights))

with open('bias1_1.csv', 'ab') as abc:
    np.savetxt(abc, initial[0][0], fmt='%.8e', delimiter=",")
with open('bias1_2.csv', 'ab') as abc:
    np.savetxt(abc, initial[1][0], fmt='%.8e', delimiter=",")

with open('weight1_1.csv', 'ab') as abc:
    np.savetxt(abc, initial[0][1], fmt='%.8e', delimiter=",")    
with open('weight1_2.csv', 'ab') as abc:
    np.savetxt(abc, initial[1][1], fmt='%.8e', delimiter=",")

In [ ]:
#Training the network
mathnet.SGD(training_data, 35, 1, .6, test_data=test_data)

In [32]:
#Write the final weights and biases to files

final = list(zip(mathnet.biases, mathnet.weights))
             
with open('bias2_1.csv', 'ab') as abc:
    np.savetxt(abc, mathnet.biases[0], fmt='%.8e', delimiter=",")
with open('bias2_2.csv', 'ab') as abc:
    np.savetxt(abc, mathnet.biases[1], fmt='%.8e', delimiter=",")

with open('weight2_1.csv', 'ab') as abc:
    np.savetxt(abc, mathnet.weights[0], fmt='%.8e', delimiter=",")    
with open('weight2_2.csv', 'ab') as abc:
    np.savetxt(abc, mathnet.weights[1], fmt='%.8e', delimiter=",")

In [45]:
#Set biases and weights to original values
mathnet.biases = ibiases
mathnet.weights = iweights